In [1]:
import import_ipynb
import DownloadingData
import Testsetsplit
import pandas as pd

importing Jupyter notebook from DownloadingData.ipynb
importing Jupyter notebook from Testsetsplit.ipynb
StratifiedShuffleSplit(n_splits=1, random_state=42, test_size=0.2,
            train_size=None)


In [2]:
housingdf=DownloadingData.load_housing_data()

In [3]:
housingdf["room_per_perhousehold"]=housingdf["total_rooms"]/housingdf["households"]
housingdf["bedrooms_per_room"]=housingdf["total_bedrooms"]/housingdf["total_rooms"]
housingdf["population_per_household"]=housingdf["population"]/housingdf["households"]

In [4]:
housingtrain=Testsetsplit.strat_train_set.drop(columns="median_house_value")
housing_labels=Testsetsplit.strat_train_set["median_house_value"]
housetraincopy=housingtrain.copy()

In [5]:
housing_labels

17606    286600.0
18632    340600.0
14650    196900.0
3230      46300.0
3555     254500.0
           ...   
6563     240200.0
12053    113000.0
13908     97800.0
11159    225900.0
15775    500001.0
Name: median_house_value, Length: 16512, dtype: float64

In [6]:
bedroom_median=housingtrain["total_bedrooms"].median()
housingtrain["total_bedrooms"].fillna(bedroom_median,inplace=True)

In [7]:
from sklearn.impute import SimpleImputer
imputer=SimpleImputer(strategy="median")#initializing imputer and setting strategy to median meaning na values to be filled by median of that column
housingtrainimputed=housingtrain.drop(columns="ocean_proximity")#imputer doesnt work on string valued columns
imputer.fit(housingtrainimputed)#computes median for columns containing empty values

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='median', verbose=0)

In [8]:
print(imputer.statistics_)
print(housingtrain.median())

[-118.51     34.26     29.     2119.5     433.     1164.      408.
    3.5409]
longitude             -118.5100
latitude                34.2600
housing_median_age      29.0000
total_rooms           2119.5000
total_bedrooms         433.0000
population            1164.0000
households             408.0000
median_income            3.5409
dtype: float64


In [9]:
X=imputer.transform(housingtrainimputed)#transforms the na values into median values and the result is a numpy array
print(X)

[[-121.89     37.29     38.     ...  710.      339.        2.7042]
 [-121.93     37.05     14.     ...  306.      113.        6.4214]
 [-117.2      32.77     31.     ...  936.      462.        2.8621]
 ...
 [-116.4      34.09      9.     ... 2098.      765.        3.2723]
 [-118.01     33.82     31.     ... 1356.      356.        4.0625]
 [-122.45     37.77     52.     ... 1269.      639.        3.575 ]]


In [10]:
housingtrain=pd.DataFrame(X,columns=housingtrainimputed.columns,index=housingtrainimputed.index)

In [11]:
from sklearn.preprocessing import OneHotEncoder
cat_encoder=OneHotEncoder()#used to provide numerical values to string dataframes(page:65)


In [12]:
housing_cat=housingdf["ocean_proximity"]

In [13]:
housing_cat.head()

0    NEAR BAY
1    NEAR BAY
2    NEAR BAY
3    NEAR BAY
4    NEAR BAY
Name: ocean_proximity, dtype: object

In [14]:
housing_tr2=housingtrain.merge(housingdf["ocean_proximity"],left_index=True,right_index=True)

In [15]:
housing_cat=housing_tr2[["ocean_proximity"]]
housing_cat1hot=cat_encoder.fit_transform(housing_cat)

In [16]:
housing_cat1hot.toarray()

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.]])

In [17]:
cat_encoder.categories_

[array(['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'],
       dtype=object)]

In [18]:
housetraincopy

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
17606,-121.89,37.29,38.0,1568.0,351.0,710.0,339.0,2.7042,<1H OCEAN
18632,-121.93,37.05,14.0,679.0,108.0,306.0,113.0,6.4214,<1H OCEAN
14650,-117.20,32.77,31.0,1952.0,471.0,936.0,462.0,2.8621,NEAR OCEAN
3230,-119.61,36.31,25.0,1847.0,371.0,1460.0,353.0,1.8839,INLAND
3555,-118.59,34.23,17.0,6592.0,1525.0,4459.0,1463.0,3.0347,<1H OCEAN
...,...,...,...,...,...,...,...,...,...
6563,-118.13,34.20,46.0,1271.0,236.0,573.0,210.0,4.9312,INLAND
12053,-117.56,33.88,40.0,1196.0,294.0,1052.0,258.0,2.0682,INLAND
13908,-116.40,34.09,9.0,4855.0,872.0,2098.0,765.0,3.2723,INLAND
11159,-118.01,33.82,31.0,1960.0,380.0,1356.0,356.0,4.0625,<1H OCEAN
